In [37]:
import numpy as np
from sklearn import preprocessing 

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter =',')

unscaled_inputs_all = raw_csv_data[:,0:-1]
targets_all = raw_csv_data[:,-1]

In [38]:
#Balance the Dataset
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i) #if the targets i is 0 and number of zeroes is bigger than number of 1s, i'll know the indices of all data points to be removed
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

In [39]:
#Standardizing the inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [40]:
#Shuffling
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [41]:
sample_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*sample_count)
validation_samples_count = int(0.1*sample_count)
test_samples_count = sample_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count] 
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]
# just to check
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1784.0 3579 0.4984632578932663
220.0 447 0.49217002237136465
233.0 448 0.5200892857142857


In [42]:
#saving the three dataset in .npz
np.savez('Audiobooks_data_train',inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

In [43]:
import tensorflow as tf

In [44]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs, train_targets  = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, valigation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

C:\Users\abhis\AppData\Local\Temp\ipykernel_21340\4265804710.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs, train_targets  = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
C:\Users\abhis\AppData\Local\Temp\ipykernel_21340\4265804710.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; f

In [45]:
#Modelling
input_size = 10 
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                           ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['Accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
#tf.keras.callbacks.EarlyStopping() configures the early stopping mechanism of the algorithm.
model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         validation_data=(validation_inputs, validation_targets),
         verbose=2)

Epoch 1/100
36/36 - 2s - loss: 0.6319 - Accuracy: 0.6301 - val_loss: 0.5183 - val_Accuracy: 0.7315 - 2s/epoch - 68ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4832 - Accuracy: 0.7455 - val_loss: 0.4535 - val_Accuracy: 0.7852 - 247ms/epoch - 7ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4348 - Accuracy: 0.7645 - val_loss: 0.4202 - val_Accuracy: 0.7897 - 212ms/epoch - 6ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4094 - Accuracy: 0.7695 - val_loss: 0.4085 - val_Accuracy: 0.7785 - 213ms/epoch - 6ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3922 - Accuracy: 0.7868 - val_loss: 0.4009 - val_Accuracy: 0.7875 - 201ms/epoch - 6ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3796 - Accuracy: 0.7899 - val_loss: 0.3880 - val_Accuracy: 0.7808 - 228ms/epoch - 6ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3717 - Accuracy: 0.7946 - val_loss: 0.3872 - val_Accuracy: 0.7875 - 216ms/epoch - 6ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3640 - Accuracy: 0.7991 - val_loss: 0.3737 - val_Accuracy: 0.7830 - 217ms/epoch - 6ms/step
Epoch 9/10

Epoch 67/100
36/36 - 0s - loss: 0.3067 - Accuracy: 0.8329 - val_loss: 0.3498 - val_Accuracy: 0.7919 - 246ms/epoch - 7ms/step
Epoch 68/100
36/36 - 0s - loss: 0.3027 - Accuracy: 0.8351 - val_loss: 0.3505 - val_Accuracy: 0.7875 - 205ms/epoch - 6ms/step
Epoch 69/100
36/36 - 0s - loss: 0.3030 - Accuracy: 0.8391 - val_loss: 0.3467 - val_Accuracy: 0.8076 - 235ms/epoch - 7ms/step
Epoch 70/100
36/36 - 0s - loss: 0.3083 - Accuracy: 0.8343 - val_loss: 0.3632 - val_Accuracy: 0.7919 - 259ms/epoch - 7ms/step
Epoch 71/100
36/36 - 0s - loss: 0.3014 - Accuracy: 0.8335 - val_loss: 0.3456 - val_Accuracy: 0.7897 - 272ms/epoch - 8ms/step
Epoch 72/100
36/36 - 0s - loss: 0.2997 - Accuracy: 0.8385 - val_loss: 0.3482 - val_Accuracy: 0.7987 - 265ms/epoch - 7ms/step
Epoch 73/100
36/36 - 0s - loss: 0.3018 - Accuracy: 0.8444 - val_loss: 0.3515 - val_Accuracy: 0.7808 - 251ms/epoch - 7ms/step
Epoch 74/100
36/36 - 0s - loss: 0.2998 - Accuracy: 0.8393 - val_loss: 0.3592 - val_Accuracy: 0.7830 - 213ms/epoch - 6ms/step


In [46]:
#testing the model 
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 4ms/step - loss: 0.3138 - Accuracy: 0.8281
